In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
figdir = "../images/"

In [ ]:
# import bayfox
# import erebusfall
# from lgmproxies.datasets.tierney import TIERNEY_REPOS
# TIERNEY_REPOS
import pandas as pd
from lgmproxies.datasets.tierney import get_repo_path
proxydata = get_repo_path("jesstierney/lgmDA")/"proxyData"
! ls {proxydata}
# df = pd.read_csv(proxydata/"Tierney2020_ProxyDataPaired.csv")
paired = pd.read_csv(proxydata/"Tierney2020_ProxyDataPaired.csv")
lh = pd.read_csv(proxydata/"Tierney2020_LHProxyData.csv")
lgm = pd.read_csv(proxydata/"Tierney2020_LGMProxyData.csv")
lgm
# get_repo_path

In [ ]:
color_map = {
    "delo": "tab:blue",
    "mg": "tab:orange",
    "tex": "tab:green",
    "uk": "tab:red"
}

marker_map = {
    "bulloides": "o",
    "incompta": "s",
    "pachy": "^",
    "ruber": "D",
    "sacculifer": "v",
    "nan": "X"
}

f, axes = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)


for ax, df in zip(axes.flat, [lh, lgm]):
    for proxytype in sorted(df["ProxyType"].unique()):
        df_proxy = df[df["ProxyType"] == proxytype]
        for species in sorted(df_proxy["Species"].unique(), key=lambda x: str(x)):
            if pd.isna(species):
                species_key = "nan"
                df_species = df_proxy[df_proxy["Species"].isna()]
                label = f"{proxytype}"
            else:
                species_key = species
                df_species = df_proxy[df_proxy["Species"] == species]
                label = f"{proxytype} ({species})"
            if len(df_species) > 0:
                ax.scatter(
                    df_species["ProxyValue"], df_species["SSTMedian"],
                    label=label,
                    color=color_map.get(proxytype, "gray"),
                    marker=marker_map.get(species_key, "x"),
                    alpha=0.6,
                    edgecolor="black",
                    linewidth=0.5
                )

    ax.set_xlabel("Proxy Value")
    if ax == axes[0]:
        ax.set_ylabel("SST Median (°C)")
    ax.set_title("LGM Proxies" if ax == axes[1] else "LH Proxies")
    ax.grid(True)
    # only show legend for the last subplot
    if ax == axes[-1]:
        ax.legend(loc="upper left", bbox_to_anchor=(1, 1), fontsize='small')

plt.tight_layout()
plt.savefig(figdir+"tierney_calibration.png", dpi=300, bbox_inches='tight')

In [ ]:
dfs = {
    "Late Holocene": lh,
    "LGM": lgm,
}

## UK 37

In [ ]:
import numpy as np

def uk37_to_sst(uk37):
    """Convert UK'37 to sea surface temperature using Müller et al. (1998)."""
    return (uk37 - 0.044) / 0.033  # SST in °C


def uk37_error(uk37):
    """Estimate standard error for UK'37 SST reconstructions."""
    return np.where(uk37 >= 0.97,  # approaching saturation
        2.0,
        1.5  # °C
    )

f, axes = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)

for i, lab in enumerate(dfs):

    df = dfs[lab]

    ax = axes[i]

    df_proxy = df[df["ProxyType"] == "uk"]

    ymedian = df_proxy["SSTMedian"]
    yerr_lower = ymedian - df_proxy["SSTLower2s"]
    yerr_upper = df_proxy["SSTUpper2s"] - ymedian
    yerr = [yerr_lower, yerr_upper]

    ax.errorbar(
        df_proxy["ProxyValue"], ymedian,
        yerr=yerr,
        fmt=marker_map["nan"],
        color=color_map["uk"],
        alpha=0.6,
        ecolor="black",
        capsize=3,
        markeredgecolor="black",
        linewidth=0.5,
        label="Tierney et al. (2020)"
    )

    ymedian = uk37_to_sst(df_proxy["ProxyValue"])
    yerr = 2 * uk37_error(df_proxy["ProxyValue"])

    ax.errorbar(
        df_proxy["ProxyValue"], ymedian,
        yerr=yerr,
        fmt=".",
        color="tab:green",
        alpha=0.6,
        ecolor="tab:green",
        capsize=3,
        # markeredgecolor="green",
        linewidth=0.5,
        label="Müller et al. (1998)",
    )

    ax.set_xlabel("UK'37 Proxy Value")
    ax.set_ylabel("SST Median (°C)")
    ax.set_title(lab)
    ax.grid(True)
    ax.legend(loc="upper left", fontsize='small')

f.suptitle("UK'37 Proxies")
plt.tight_layout()
plt.savefig(figdir+"uk37_calibration.png", dpi=300, bbox_inches='tight')

## Tex 86

In [ ]:
import numpy as np

def tex86_to_sst(tex86):
    """Convert TEX86 to sea surface temperature (SST) using Kim et al. (2010)."""
    return 68.4 * tex86 - 38.6  # SST in °C

def tex86_error(tex86):
    """Return standard error for TEX86-derived SST (Kim et al. 2010)."""
    return 2.5  # °C, can be regionally adjusted

f, axes = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)

for i, lab in enumerate(dfs):

    df = dfs[lab]
    ax = axes[i]

    df_proxy = df[df["ProxyType"] == "tex"]


    ymedian = df_proxy["SSTMedian"]
    yerr_lower = ymedian - df_proxy["SSTLower2s"]
    yerr_upper = df_proxy["SSTUpper2s"] - ymedian
    yerr = [yerr_lower, yerr_upper]

    ax.errorbar(
        df_proxy["ProxyValue"], ymedian,
        yerr=yerr,
        fmt=marker_map["nan"],
        color=color_map["tex"],
        alpha=0.6,
        ecolor="black",
        capsize=3,
        markeredgecolor="black",
        linewidth=0.5,
        label="Tierney et al. (2020)"
    )

    ymedian = tex86_to_sst(df_proxy["ProxyValue"])
    yerr = 2 * tex86_error(df_proxy["ProxyValue"])

    ax.errorbar(
        df_proxy["ProxyValue"], ymedian,
        yerr=yerr,
        fmt=".",
        color="tab:green",
        alpha=0.6,
        ecolor="tab:green",
        capsize=3,
        # markeredgecolor="green",
        linewidth=0.5,
        label="Kim et al. (2010) (Chat GPT...)",
    )

    ax.set_xlabel("Proxy Value")
    ax.set_ylabel("SST Median (°C)")
    ax.set_title(lab)
    ax.grid(True)
    ax.legend(loc="upper left", fontsize='small')

f.suptitle("Tex'86 Proxies")
f.tight_layout()
f.savefig(figdir+"tex86_calibration.png", dpi=300, bbox_inches='tight')

## $\Delta^{18}O$

In [ ]:
import numpy as np

def delta18o_to_temp(delta18o_c, delta18o_sw=0.0):
    """
    Convert δ18O of calcite (‰ VPDB) to temperature (°C).
    delta18o_sw: seawater δ18O (‰ VSMOW, converted to VPDB by -0.27‰).
    """
    delta = delta18o_c - (delta18o_sw - 0.27)  # match standards
    return 16.9 - 4.38 * delta + 0.1 * delta**2  # °C

def delta18o_error():
    """Estimate uncertainty in δ18O paleotemperature."""
    return 1.5  # °C typical if δ18O_sw is estimated reasonably


f, axes = plt.subplots(5, 2, figsize=(12, 5*4), sharex=True, sharey=True)

for j, lab in enumerate(dfs):
    df = dfs[lab]
    df_proxy = df[df["ProxyType"] == "delo"]

    for i, species in enumerate(sorted(df_proxy["Species"].unique(), key=lambda x: str(x))):

        ax = axes[i, j]

        df_species = df_proxy[df_proxy["Species"] == species]
        ymedian = df_species["SSTMedian"]
        yerr_lower = ymedian - df_species["SSTLower2s"]
        yerr_upper = df_species["SSTUpper2s"] - ymedian
        yerr = [yerr_lower, yerr_upper]

        ax.errorbar(
            df_species["ProxyValue"], ymedian,
            yerr=yerr,
            fmt=marker_map[species],
            color=color_map["delo"],
            alpha=0.6,
            ecolor="black",
            capsize=3,
            markeredgecolor="black",
            linewidth=0.5,
            label="Tierney et al. (2020)"
        )

        # ymedian = uk37_to_sst(df_proxy["ProxyValue"])
        # yerr = 2 * uk37_error(df_proxy["ProxyValue"])

        # ax.errorbar(
        #     df_proxy["ProxyValue"], ymedian,
        #     yerr=yerr,
        #     fmt=".",
        #     color="tab:green",
        #     alpha=0.6,
        #     ecolor="tab:green",
        #     capsize=3,
        #     # markeredgecolor="green",
        #     linewidth=0.5,
        #     label="Müller et al. (1998)",
        # )

        ax.set_xlabel("Proxy Value")
        ax.set_ylabel("SST Median (°C)")
        ax.set_title(f"{lab} {species}")
        ax.grid(True)
        ax.legend(loc="upper left", fontsize='small')

f.suptitle(r"$\Delta^{18}O$ Proxies", fontsize=16)
f.tight_layout()
f.savefig(figdir+"delo_calibration.png", dpi=300, bbox_inches='tight')

## Mg / Ca

In [ ]:

f, axes = plt.subplots(4, 2, figsize=(12, 4*4), sharex=True, sharey=True)

for j, lab in enumerate(dfs):
    df = dfs[lab]
    df_proxy = df[df["ProxyType"] == "mg"]

    for i, species in enumerate(sorted(df_proxy["Species"].unique(), key=lambda x: str(x))):
        ax = axes[i, j]

        df_species = df_proxy[df_proxy["Species"] == species]
        ymedian = df_species["SSTMedian"]
        yerr_lower = ymedian - df_species["SSTLower2s"]
        yerr_upper = df_species["SSTUpper2s"] - ymedian
        yerr = [yerr_lower, yerr_upper]

        ax.errorbar(
            df_species["ProxyValue"], ymedian,
            yerr=yerr,
            fmt=marker_map[species],
            color=color_map["mg"],
            alpha=0.6,
            ecolor="black",
            capsize=3,
            markeredgecolor="black",
            linewidth=0.5,
            label="Tierney et al. (2020)"
        )

        # ymedian = uk37_to_sst(df_proxy["ProxyValue"])
        # yerr = 2 * uk37_error(df_proxy["ProxyValue"])

        # ax.errorbar(
        #     df_proxy["ProxyValue"], ymedian,
        #     yerr=yerr,
        #     fmt=".",
        #     color="tab:green",
        #     alpha=0.6,
        #     ecolor="tab:green",
        #     capsize=3,
        #     # markeredgecolor="green",
        #     linewidth=0.5,
        #     label="Müller et al. (1998)",
        # )

        ax.set_xlabel("Proxy Value")
        ax.set_ylabel("SST Median (°C)")
        ax.set_title(f"{lab} {species}")
        ax.grid(True)
        ax.legend(loc="upper left", fontsize='small')

f.suptitle(r"Mg / Ca Proxies", fontsize=16)
f.tight_layout()
f.savefig(figdir+"mg_calibration.png", dpi=300, bbox_inches='tight')